In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
retail_df = pd.read_csv("/content/drive/MyDrive/colab/Data Science/dataset-3.csv",delimiter=",")
retail_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010/12/1 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010/12/1 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010/12/1 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010/12/1 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010/12/1 08:26,3.39,17850.0,United Kingdom


### Preprocessing  
There is a little cleanup, we need to do. First, some of the descriptions have spaces that need to be removed. We’ll also drop the rows that don’t have invoice numbers and remove the credit transactions (those with invoice numbers containing C).

In [4]:
#remove some of descriptions have spaces.
retail_df['Description'] = retail_df['Description'].str.strip()

#remove NA values
retail_df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)

#remove cancelled orders
retail_df['InvoiceNo'] = retail_df['InvoiceNo'].astype('str')
retail_df = retail_df[~retail_df['InvoiceNo'].str.contains('C')]

### France

In [5]:
#choose country
retail_df_france = retail_df[retail_df['Country'] == 'France']

In [17]:
#Create a shopping basket, record what items are included in each order, and the number of items
basket = (retail_df_france.groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))
basket

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580986,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581171,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
#Convert an order's item to with and without purchases
def encode_units(x):
  if x <= 0:
    return 0
  if x >= 1:
    return 1
#Convert each data in the basket
basket_sets = basket.applymap(encode_units)
#Delete POSTAGE
basket_sets.drop('POSTAGE',inplace=True, axis=1)

Support: The probability of this product combination appearing in all purchase orders  
Confidence: After purchasing product A, the probability of purchasing product B  
Lift: Determine the degree of correlation between product A and product B.  
*   If Lift is less than 1, it means a negative correlation,
*   if Lift is equal to it means no correlation.   
*   if Lift is greater than 1, it means a positive correlation.

支持度（Support）：這個商品組合在所有購買清單中，出現的機率。  
信賴度（Confidence）：購買A商品後，購買B商品的機率。  
提昇度（Lift）：判斷A商品與B商品的相關程度，若小於1表示負相關，等於1表示完全無相關，大於1表示正相關。

In [35]:
#only use the product combination with support > 0.05
frequent_itemsets = apriori(basket_sets, min_support=0.05 , use_colnames=True)
frequent_itemsets.head()

,support,itemsets
0,0.071429,(4 TRADITIONAL SPINNING TOPS)
1,0.096939,(ALARM CLOCK BAKELIKE GREEN)
2,0.102041,(ALARM CLOCK BAKELIKE PINK)
3,0.094388,(ALARM CLOCK BAKELIKE RED)
4,0.068878,(ASSORTED COLOUR MINI CASES)


According to lift, the correlation between PACK OF 6 SKULL PAPER CUPS and PACK OF 6 SKULL PAPER PLATES is very strong  
According to confidence, if you buy PACK OF 6 SKULL PAPER CUPS, you have an 80% chance of buying PACK OF 6 SKULL PAPER PLATES.  
Therefore, PACK OF 6 SKULL PLATES can be recommended to those customers who have purchased PACK OF 6 SKULL PAPER CUPS  

根據lift，PACK OF 6 SKULL PAPER CUPS與PACK OF 6 SKULL PAPER PLATES的關聯性非常強  
根據confidence，如果購買PACK OF 6 SKULL PAPER CUPS，則購買PACK OF 6 SKULL PAPER PLATES機率有80%。  
因此，可以推薦PACK OF 6 SKULL PLATES給那些有購買PACK OF 6 SKULL PAPER CUPS的顧客  

In [34]:
#frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))

# Find the products combination with confidence > 0.7
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)
rules.sort_values('lift',ascending = False).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
9,(PACK OF 6 SKULL PAPER CUPS),(PACK OF 6 SKULL PAPER PLATES),0.063776,0.056122,0.051020,0.800000,14.254545,0.047441,4.719388
10,(PACK OF 6 SKULL PAPER PLATES),(PACK OF 6 SKULL PAPER CUPS),0.056122,0.063776,0.051020,0.909091,14.254545,0.047441,10.298469
6,(CHILDRENS CUTLERY DOLLY GIRL),(CHILDRENS CUTLERY SPACEBOY),0.071429,0.068878,0.063776,0.892857,12.962963,0.058856,8.690476
7,(CHILDRENS CUTLERY SPACEBOY),(CHILDRENS CUTLERY DOLLY GIRL),0.068878,0.071429,0.063776,0.925926,12.962963,0.058856,12.535714
19,"(ALARM CLOCK BAKELIKE PINK, ALARM CLOCK BAKELI...",(ALARM CLOCK BAKELIKE RED),0.073980,0.094388,0.063776,0.862069,9.133271,0.056793,6.565689


In this part ,I figured out what product the customer would buy if they bought 2 products, and set the conditions of confidence > 0.7 and lift > 2.  
  
According to lift, ALARM CLOCK BAKELIKE GREEN is very closely related to ALARM CLOCK BAKELIKE PINK
According to confidence, if you buy ALARM CLOCK BAKELIKE GREEN and ALARM CLOCK BAKELIKE PINK, you have an 86% chance of buying ALARM CLOCK BAKELIKE RED.
Therefore, ALARM CLOCK BAKELIKE RED can be recommended to those customers who have purchased ALARM CLOCK BAKELIKE GREEN and ALARM CLOCK BAKELIKE *PINK*

我找出了購買2種產品的情況下，顧客會購買什麼產品，並設定confidence>0.7 和 lift > 2的條件。  

根據lift，ALARM CLOCK BAKELIKE GREEN與ALARM CLOCK BAKELIKE PINK的關聯性非常強  
根據confidence，如果購買ALARM CLOCK BAKELIKE GREEN與ALARM CLOCK BAKELIKE PINK，則購買ALARM CLOCK BAKELIKE RED 機率有86%。  
因此，可以推薦ALARM CLOCK BAKELIKE RED給那些有購買ALARM CLOCK BAKELIKE GREEN與ALARM CLOCK BAKELIKE PINK的顧客  

In [40]:
rules["antecedant_len"] = rules["antecedents"].apply(lambda x: len(x))
rules[ (rules['antecedant_len'] >= 2) &
       (rules['confidence'] > 0.7) &
       (rules['lift'] > 2) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedant_len
19,"(ALARM CLOCK BAKELIKE PINK, ALARM CLOCK BAKELI...",(ALARM CLOCK BAKELIKE RED),0.073980,0.094388,0.063776,0.862069,9.133271,0.056793,6.565689,2
20,"(ALARM CLOCK BAKELIKE PINK, ALARM CLOCK BAKELI...",(ALARM CLOCK BAKELIKE GREEN),0.073980,0.096939,0.063776,0.862069,8.892922,0.056604,6.547194,2
21,"(ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...",(ALARM CLOCK BAKELIKE PINK),0.079082,0.102041,0.063776,0.806452,7.903226,0.055706,4.639456,2
22,"(PLASTERS IN TIN SPACEBOY, PLASTERS IN TIN CIR...",(PLASTERS IN TIN WOODLAND ANIMALS),0.089286,0.170918,0.068878,0.771429,4.513433,0.053617,3.627232,2
23,"(SET/6 RED SPOTTY PAPER PLATES, SET/20 RED RET...",(SET/6 RED SPOTTY PAPER CUPS),0.102041,0.137755,0.099490,0.975000,7.077778,0.085433,34.489796,2
24,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...",(SET/6 RED SPOTTY PAPER PLATES),0.102041,0.127551,0.099490,0.975000,7.644000,0.086474,34.897959,2
25,"(SET/6 RED SPOTTY PAPER PLATES, SET/6 RED SPOT...",(SET/20 RED RETROSPOT PAPER NAPKINS),0.122449,0.132653,0.099490,0.812500,6.125000,0.083247,4.625850,2


### conclusion
With this kind of analysis from the field of mareting you can now determine which products are most often bought in combination with each other. With this knowledge it is possible to arrange the products efficiently in the store. In the best case, products that are often bought together are positioned in the opposite direction in the store so that customers are forced to walk past as many other products as possible.

Furthermore, one can now consider targeted discount campaigns. If you discount a product that is often bought in combination with others, you increase the chance of buying these products in combination, whereby a small discount is granted on only one.

### 結論  
通過市場行銷領域的這種分析，可以確定哪些產品最常被組合購買。並且可以利用這些資訊在商店中有效地安排產品的位置。我們可以試著將經常一起購買的產品在商店中的位置相反，這樣顧客就不得不經過其他產品。

此外，也可以考慮有針對一些產品的推出折扣活動。如果對經常與其他產品組合購買的產品打折，則可以增加購買這些產品的機會。